In [1]:
import os
os.chdir('../../..')
print(os.getcwd()) # Ensure this is you Amphion root path, otherwise change the above path to you amphion root path
import sys
sys.path.append('.')

/home/lijiaqi/release/AmphionVALLEv2


In [2]:
# put your cheackpoint file (.bin) in the root path of AmphionVALLEv2
# or use your own pretrained weights
ar_model_path = 'ckpts/valle_ar_mls_196000.bin'  # download the file on https://huggingface.co/jiaqili3/vallex/resolve/main/valle_ar_mls_encodec.bin?download=true
nar_model_path = 'ckpts/valle_nar_mls_164000.bin'  # download the file on https://huggingface.co/jiaqili3/vallex/resolve/main/valle_nar_mls_encodec.bin?download=true
speechtokenizer_path = 'ckpts/speechtokenizer_hubert_avg' # https://hf-mirror.com/fnlp/SpeechTokenizer/tree/main/speechtokenizer_hubert_avg
# ar_model_path = './ckpt/valle_gpt_simple/ar_libritts_dev_clean/checkpoint/train-dev-clean-epoch-0843_step-0156000_loss-0.000000/pytorch_model.bin' 
# nar_model_path = './ckpt/valle_gpt_simple/nar_libritts_dev_clean/checkpoint/epoch-0021_step-0004000_loss-5.246324/pytorch_model.bin'

In [3]:
from models.tts.valle_v2.valle_inference import ValleInference
# change to device='cuda' to use CUDA GPU for fast inference
# change "use_vocos" to True would give better sound quality
# If you meet problem with network, you could set "use_vocos=False", though would give bad quality
model = ValleInference(ar_path=ar_model_path, nar_path=nar_model_path, speechtokenizer_path=speechtokenizer_path, device='cpu')
# model = ValleInference(use_vocos=False, ar_path=ar_model_path, nar_path=nar_model_path, device='cuda')

/mntnfs/lee_data1/tzeying/anaconda/envs/amphion/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mntnfs/lee_data1/tzeying/anaconda/envs/amphion/lib/python3.9/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


ImportError: cannot import name 'DataLoaderConfiguration' from 'accelerate' (/mntnfs/lee_data1/tzeying/anaconda/envs/amphion/lib/python3.9/site-packages/accelerate/__init__.py)

In [ ]:
# prepare inference data
import librosa
import torch
wav, _ = librosa.load('./egs/tts/valle_v2/prompt_examples/example.wav', sr=16000)
wav = torch.tensor(wav, dtype=torch.float32)
from IPython.display import Audio
Audio(wav, rate = 16000)

In [ ]:
# The transcript of the prompt part
prompt_transcript_text = 'and keeping eternity before the eyes'

# Here are the words you want the model to output
target_transcript_text = 'It presents a unified framework that is inclusive of diverse generation tasks and models with the added bonus of being easily extendable for new applications'
from models.tts.valle_v2.g2p_processor import G2pProcessor
g2p = G2pProcessor()
prompt_transcript = g2p(prompt_transcript_text, 'en')[1]
target_transcript = g2p(target_transcript_text, 'en')[1]

In [ ]:
prompt_transcript = torch.tensor(prompt_transcript).long()
target_transcript = torch.tensor(target_transcript).long()
transcript = torch.cat([prompt_transcript, target_transcript], dim=-1)
batch = {
        'speech': wav.unsqueeze(0),
        'phone_ids': transcript.unsqueeze(0),
}

In [ ]:
# print the contents of the model input
# `phone_ids` contains a concatenation of `prompt_transcript` and `target_transcript` 
batch

{'speech': tensor([[ 3.0518e-05,  3.0518e-05,  3.0518e-05,  ..., -3.0518e-05,
          -3.0518e-05,  3.0518e-05]]),
 'phone_ids': tensor([[  5,  28, 149,  72, 219, 134, 127, 170, 115, 147, 219, 113, 185,  91,
          149,  30, 185, 123, 219,  65, 115, 106,  43, 172, 219,  73,  29, 219,
           59, 214,   6,   5, 116, 181, 219, 168, 173, 124, 218,  82, 149, 185,
          175, 219,  28, 219, 210, 200, 149,  30, 106,  64,  72, 219, 104, 173,
          100, 143, 209,  94, 135, 219,  73,  24, 181, 219, 116, 214, 219, 113,
          149, 136, 140, 200, 179, 115, 205, 219,  31, 205, 219,  71,  58, 206,
           91, 175, 219, 131,  85, 149,  88, 100, 178,  30, 145, 219, 180,  24,
          179, 136, 175, 219,  28, 149,  72, 219, 141,  15,  76,  30, 140, 214,
          219, 207, 118,  74, 219,  73,  29, 219,  22,  76,  30,  72, 219,  65,
          155, 149,  30, 175, 219,  31, 205, 219,  65, 127, 115, 147, 219, 125,
          218,  30, 140, 123, 219,  83, 136, 179, 185,  82, 149,  76, 

In [ ]:
configs = [dict(
    top_p=0.9,
    top_k=5,
    temperature=0.95,
    repeat_penalty=1.0,
    max_length=2000,
    num_beams=1,
)] # model inference hyperparameters
output_wav = model(batch, configs)

In [ ]:
output_wav # The output wav is a tensor of shape [1,1,T]

tensor([[[-3.9004e-05, -4.0838e-05, -1.0060e-05,  ...,  6.2591e-04,
          -4.9415e-04, -1.3975e-03]]])

In [ ]:
print(f'prompt_transcript : {prompt_transcript_text}')
print(f'target_transcript : {target_transcript_text}')
Audio(output_wav.squeeze(0), rate = 16000)

prompt_transcript : and keeping eternity before the eyes
target_transcript : It presents a unified framework that is inclusive of diverse generation tasks and models with the added bonus of being easily extendable for new applications


In [ ]:
import torchaudio
torchaudio.save('out.wav', output_wav.squeeze(0), 24000)